In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import tqdm

## Random walk and diffusion

**Why are we doing this:** The example in this notebook -- random walk between a finite set of states or points -- becomes the diffusion equation in the limit as the number of states grows very large.
It'll enable us to work out all the hard parts of discretizing diffusive-type problems in time first, without having to worry about space discretization.
That'll come later.

Our first example is diffusive transport.
Diffusion shows up in all manner of settings ranging from from ecology to hillslope evolution.
What we see at the macro-scale as diffusion often emerges from what is, at the micro-scale, a *random walk*.
To pick a concrete example, we'll start by thinking about a rare isotope embedded in a crystal lattice.
For example, you could think of a water molecule containing an ${}^{18}$O atom frozen into an ice core.
We imagine that this molecule could reside at one of $N$ sites in the crystal lattice.
This molecule will periodically jump from its current position to one of the neighboring sites on the lattice.
To characterize what happens, we need to know (1) where does the molecule start? (2) what is the exepcted *holding time* $\tau$ before the molecule will jump to another site? and (3) what are the neighboring sites and what is the probability of jumping to each site?

We imagine a one-dimensional crystal lattice to start off, so the neighbors of site $k$ are $k + 1$ and $k - 1$.
We suppose for now that the probabilities of going up or down are equal.
Finally, we suppose that we know what the holding time $\tau$ is.
To simulate a sample trajectory of the particle, we proceed by:
1. Generate an *exponential* random variable with intensity $\tau$.
2. Pick up or down with probability 1/2.

The simplest setting of this problem assumes equal probability to go up as well as down.
We can also look at situations where the isotope is more likely to go one direction than the other.
This asymmetry is known as random walk with *drift*.
We could also see what happens if the holding times vary within the lattice.
I've used a 1D lattice so far because it's convenient.
You could also imagine all of this taking place in higher dimensions.
The algorithm for generating sample paths in these more complicated cases is essentially the same as what I described above.
What we need are:
$$\begin{align}
    \tau_k & = \text{ holding time in state } k \\
    \rho_{k\to\ell} & = \text{ probability of transitioning from $k$ to $\ell$}.
\end{align}$$
We'll require that
$$\sum_\ell \rho_{k\to\ell} = 1$$
for every state $k$, i.e. the sum of the transition probabilities to all other states is 1.

#### Generating a sample path

Now let's write some code so we can explore these ideas.
The function below will generate a single sample path of a random walk.
I've left some parts blank with comments.
Fill them in and run the sample code below to see if it worked.

In [ ]:
def generate_sample_path(
    starting_state: int,
    holding_times: np.ndarray,
    probabilities: np.ndarray,
    final_time: float,
    rng: np.random.Generator,
):
    r"""Generate a single sample path of the continuous-time Markov chain

    Parameters
    ----------
    starting_state:
        The initial state or position of the Markov chain
    holding_times:
        How long the Markov chain spends on average in each state
    probabilities:
        A matrix such that `probabilities[i, j]` describes the probability of
        transition from state `i` to state `j`
    final_time:
        How long to simulate for
    rng:
        A random number generator object

    Returns
    -------
    states, times: np.ndarray, np.ndarray
        Arrays describing the times of transitions and what state the system is
        in after each transition
    """
    i = starting_state
    states, times = [i], [0.0]
    while times[-1] < final_time:
        dt = rng.exponential(holding_times[i])
        neighbors, = probabilities[i, :].nonzero()
        i = rng.choice(neighbors, p=probabilities[i, neighbors])
        times.append(times[-1] + dt)
        states.append(i)

    return np.array(states), np.array(times)

In [ ]:
rng = np.random.default_rng(seed=1729)

We'll start with a system consisting of 20 different possible states.
Create two arrays, `holding_times` and `probabilities`.
The holding times array will have shape `num_states` and will all be uniformly equal to 1.
The `probabilities` array should have shape `(num_states, num_states)`.
Make the probability of jumping from state `k` to `k + 1` equal to 0.45, and the probability of jumping from state `k` to `k - 1` equal to 0.55.
We then have to do something special at the endpoints.
Make the probability equal to 1 for jumping from state 0 to state 1, and for jumping from the last state to the second-to-last state.
Make the holding time at the bottom (index \#0) 2.0 instead of 1.0.

In [ ]:
num_states = 20
holding_times = ...
probabilities = ...

The helper function below plots a single sample path.

In [ ]:
def show_sample_path(xs, ts, ax, color="tab:blue", **kwargs):
    for t1, t2, x1, x2 in zip(ts[:-1], ts[1:], xs[:-1], xs[1:]):
        ax.plot([t1, t2], [x1, x1], color=color, **kwargs)
        ax.plot([t2, t2], [x1, x2], color=color, **kwargs)

And this will plot a few sample paths.

In [ ]:
rng = np.random.default_rng(seed=1729)

starting_state = num_states // 2
final_time = 100.0
args = (starting_state, holding_times, probabilities, final_time, rng)

fig, ax = plt.subplots()
for color in ["tab:blue", "tab:green", "tab:orange"]:
    xs, ts = generate_sample_path(*args)
    show_sample_path(xs, ts, ax, color=color, alpha=0.5)
ax.set_xlabel("time")
ax.set_ylabel("state")
ax.set_xlim((0, final_time))
ax.set_ylim((0, num_states));

**Exercise:** Can you think of other physical systems that are random walks?

#### Many sample paths

Now suppose we were to look at a very large number of realizations or sample paths of this process.
We can summarize its behavior by looking at the time-evolving probability mass
$$p_k(t) = \text{fraction of sample paths that are at state $k$ at time $t$}.$$
We can pack all of these into a vector $p(t) = \{p_1(t), \ldots, p_N(t)\}$.
**Can we describe how $p$ evolves in time?**

Before we try to answer that question, let's generate a whole lot more sample paths and see what the density looks like at a bunch of snapshots in time.
The code below is a helper function that will take a partially computed probability density $p$ at several times and a single sample path, and add that path's contribution into the density function.

In [ ]:
def sum_density(
    density: np.ndarray,
    times: np.ndarray,
    states: np.ndarray,
    final_time: float,
):
    num_times = density.shape[1]
    dt = final_time / num_times
    for t1, t2, state in zip(times[:-1], times[1:], states[:-1]):
        for index in range(int(t1 / dt), min(int(t2 / dt), num_times)):
            density[state, index] += 1

Next, we'll generate 1000 sample paths, all of them starting from the same point, and fill up the empirical density.
Here I'm using a package called tqdm so that we can see a progress bar.
If you're doing long-running computations, this can be really helpful.

In [ ]:
num_times = 100
density = np.zeros((num_states, num_times))

num_trials = 1000
for trial in tqdm.trange(num_trials):
    xs, ts = generate_sample_path(*args)
    sum_density(density, ts, xs, final_time)

Let's look at the results.
Observe how the states tend to cluster near the bottom.
Makes sense right?

In [ ]:
fig, ax = plt.subplots()
ax.imshow(density / num_trials, origin="lower")
ax.set_xlabel("time")
ax.set_ylabel("state");

Now we'll zoom in on the last half.

In [ ]:
fig, ax = plt.subplots()
ax.imshow(density[:, num_times // 2:] / num_trials, origin="lower");

#### Density evolution

Ok that was neat I suppose.
Can we describe how the probability density would evolve if we were able to take infinitely many trials?

We can observe that the rate of change of probability in $p_k$ is the difference of two terms.
Probability can flow into $k$ from other states $\ell$, and probability can flow out of $k$ into other states $\ell$.
The rates are determined by the holding times and transition probabilities:
$$\dot p_k = \underbrace{\sum_\ell\frac{\rho_{\ell\to k}}{\tau_\ell}p_\ell}_{\text{flux in}} - \underbrace{\sum_\ell\frac{\rho_{k\to\ell}}{\tau_k}p_k}_{\text{flux out}}$$
We can turn all of this into linear algebra.
We define the *generator* matrix $A$ as
$$A = \left(I - \rho^*\right)\text{diag}(\tau)^{-1}$$
and this describes how the probability evolves as
\begin{equation}
    \dot p = -A\, p.
\end{equation}
Remember that all the transition probabilities (the rows of $\rho$) have to add up to 1.
What this means is -- and you might want to try some examples -- that if we take the transpose of $A$ and multiply it by all 1s, we get 0:
\begin{equation}
    A^*\mathbf 1 = 0.
\end{equation}
This means that
\begin{equation}
    \det A^* = \det A = 0,
\end{equation}
so there must be some other non-zero vector $p_{\text{eq}}$ such that $A\cdot p_{\text{eq}} = 0$.
This special vector $p_{\text{eq}}$ is the *equilibrium* density.
In the computational exercises, you'll try a bunch of cases and see how things evolve.

Fill in the body of the function below to compute the rate matrix from the holding times and transition probabilities.
Remember how to compute transposes of arrays, how to form the identity matrix, and how to [form a diagonal matrix](https://numpy.org/doc/stable/reference/generated/numpy.diag.html) from a vector.

In [ ]:
def make_rate_matrix(holding_times, probabilities):
    ...

In [ ]:
A = make_rate_matrix(holding_times, probabilities)

Compute the eigendecomposition of the rate matrix $A$ and save the results in two variables `λ`, `V`.
Print out the eigenvalues.
If you formed the rate matrix correctly, the eigenvalues should all be real or have negligible imaginary parts, there should be one eigenvalue very close to zero (that's the equilibrium density), and the remaining eigenvalues should all be positive.

The eigendecomposition routines in numpy and scipy don't give any guarantees about what order the eigenvalues and eigenvectors are in.
The function [np.argsort](https://numpy.org/doc/stable/reference/generated/numpy.argsort.html) will return a list of integer indices describing how to permute an array so that it's in order.
Redefine the arrays of eigenvalues and eigenvectors using this permutation.

In [ ]:
permutation = np.argsort(np.abs(λs))
λ = λ[permutation]
V = V[:, permutation]

The scatter plot below will show all the eigenvalues; if you permuted them correctly, you should see them getting larger to the right.

In [ ]:
fig, ax = plt.subplots()
ax.scatter(np.linspace(0, num_states, num_states), λ.real);

The eigendecomposition algorithm also doesn't give any guarantees about the normalization of the eigenvectors.
For example, you and I know that the eigenvector corresponding to the eigenvalue $\lambda = 0$ represents the equilibrium probability density $p_{eq}$, so all of its entries should be positive and the total probability should be equal to 1 ($\mathbf{1}^*p_{eq} = 1$).
But we can multiply an eigenvector of a matrix by any non-zero number and it's still an eigenvector.
Define a variable `p_eq` to be the 0th eigenvector of $A$, normalize the result to 1, and plot it.
What do you observe about it and does it make sense given the rules that we created for how the system evolves?

In [ ]:
p_eq = ...

The eigenvalues of $A$ have units of inverse time, so their reciprocals are natural timescales for the problem.
The 0th eigenvalue of $A$ is close to 0, so its reciprocal is basically infinity.
If we take the reciprocal of the next eigenvalue (index \#1) and the last eigenvalue, these will give us respectively the time for the longest and shortest modes to decay.
Compute these timescales, save them in variables `longest_timescale` and `shortest_timescale`, and print out their values.
Remember that you can get the last element of an array `q` with `q[-1]`.

Now we'll compute the evolution of the probability density, which we'll store in a variable called `ps`.
One index will represent states and another index will be time.
But we have a choice -- should the shape be `(num_steps + 1, num_states)` or should it be `(num_states, num_steps + 1)`?
The answer is: yes.
We'll go with the first choice `(num_steps + 1, num_states)`.
If we want to peel off the probability density at time index `n`, we can do this with `ps[n, :]`, but numpy also gives us the shorthand `ps[n]` which is particularly convenient.
There are also performance reasons for doing this.

Create an zero array `ps` of shape `(num_steps + 1, num_states)`.
Then set `ps[0, starting_state]` to be equal to 1, representing our initial state.

In [ ]:
num_steps = 256
ps = ...

We'll make our simulation run for twice the longest timescale that you computed above.
Save the duration `dt` of one timestep; this should be the final time divided by the number of steps.

In [ ]:
dt = ...

In the previous notebook, we computed the application of the matrix exponential using a formula expressed in terms of the eigenvalues and eigenvectors of $A$.
The scipy package, which builds on many of the routines in numpy, has a routine to compute the matrix exponential directly.
If we want to compute
$$p(n\cdot dt) = \exp(-n\cdot dt\cdot A)p_0,$$
we can also do
$$p(n\cdot dt) = \underbrace{\exp(-dt\cdot A)\cdot\ldots\cdot \exp(-dt\cdot A)}_{n\text{ times}}p_0.$$
(Why can we do this?)
But if we already computed $p((n - 1)dt)$, then we don't have to compute a bunch of matrix powers -- we can just do
$$ps(n\cdot dt) = \exp(-dt\cdot A)p((n - 1)dt).$$
The upshot of all this is that we can compute the matrix
$$G = \exp(-dt\cdot A)$$
once and use it over and over again.
Compute this matrix below.

Now write a loop to fill in the array `ps`.

In [ ]:
for n in range(num_steps):
    ...

If you did everything right, this should make a movie of the result.

In [ ]:
%%capture
fig, ax = plt.subplots()
line, = ax.plot(ps[0])
def animate(p):
    line.set_ydata(p)
animation = FuncAnimation(fig, animate, ps, interval=1e3/60)

In [ ]:
HTML(animation.to_jshtml())

To wrap things up, let's look at the projection of the probability density at each time onto 0th, 1th, and last eigenvectors of $A$.
I've normalized them so that the magnitudes of both are the same at the start.
Notice how the one mode decays much faster than the other.
What do you think happens to this contrast in timescales when we take more and more states?

In [ ]:
projections = np.linalg.solve(V, ps.T).T
ts = np.linspace(0, 2 * longest_timescale, num_steps + 1)

fig, ax = plt.subplots()
ax.set_xlabel("time")
ax.plot(ts, abs(projections[:, 1] / projections[0, 1]), label="longest")
ax.plot(ts, abs(projections[:, -1] / projections[0, -1]), label="shortest")
ax.legend();